## Cuda

A general purpose parallel computing platform and programming model

<img align="right"  src="https://i.ibb.co/JQGz39k/cuda1.png" alt="Drawing" style="width: 500px;"/>

<div style="text-align: left"> 

    Grid: Group of blocks executing a kernel
    One grid per CUDA kernel
    
    Block: Group of threads that can be scheduled independently
    Max threads in a block: 1024
    
    Thread: A single context of execution
    
    Kernel: Function that will execute in parallel on multiple threads

</div>

<img src="https://i.ibb.co/jkyFsvc/cuda2.png" alt="Drawing" style="width: 700px;"/>

<div style="text-align: left"> 

    CUDA Memory Hierarchy:
    
    Per thread local memory: Available only to the single thread
    
    Block shared memory: Shared by all the threads in a block
        Faster than global memory
    
    Global memory: Shared by all blocks and all grids

</div>

<img align="left"  src="https://i.ibb.co/hFp1yTc/cuda3.png" alt="Drawing" style="width: 400px;"/>


## Global memory access

<img src="https://i.ibb.co/rQ4sLvg/matmul1.png" alt="Drawing" style="width: 800px;"/>

In [1]:
from numba import cuda
print(cuda.gpus)

<Managed Device 0>


## A more complex example: matrix multiplication

The following code sample is a straightforward implementation of matrix multiplication for matrices where each thread reads one row of A and one column of B and computes the corresponding element of C. For input arrays where A.shape == (m, n) and B.shape == (n, p) then the result shape will be C.shape = (m, p).

In [2]:
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    C is for saving output
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

The host code must create and initiliaze the A and B arrays, then move them to the device. Next, it must allocate space on the device for the result array. Once the kernel has completed, the result array must be copied back to the host so that it can be displayed.

Create a program called cuda2.py using the code below to see how the kernel works for the input arrays. Notice that the number of threads per block and blocks per grid is not really important, other than to ensure that there are enough threads to complete the calculation.

In [16]:
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        
# Host code

# Initialize the data arrays
A = numpy.full((32, 48), 3, float) # matrix containing all 3's
B = numpy.full((48, 16), 4, float) # matrix containing all 4's

# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.device_array((32, 16))    
# > create an empty array in device for saving output

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)
# > use ceil, it's better to have extra empty space


# Start the kernel 
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
# > must include C_global_mem because CUDA cannot return values

# Copy the result back to the host
C = C_global_mem.copy_to_host()

print(C)

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[[576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576

A problem with this code is that each thread is reading from the global memory containing the copies of A and B. In fact, the A global memory is read B.shape[1] times and the B global memory is read A.shape[0] times. Since global memory is fairly slow, this results in an inefficient use of the GPU.

### Shared memory and thread synchronization
A limited amount of shared memory can be allocated on the device to speed up access to data, when necessary. That memory will be shared (i.e. both readable and writable) amongst all threads belonging to a given block and has faster access times than regular device memory. It also allows threads to cooperate on a given solution. You can think of it as a manually-managed data cache.

The function to create a shared memory array is:

```shared_array = cuda.shared.array(shape,type)```

**shape**

    is either an integer or a tuple of integers representing the array’s dimensions.

**type**

    is a Numba type of the elements needing to be stored in the array
    The memory is allocated once for the duration of the kernel, unlike traditional dynamic memory management.

Because the shared memory is a limited resource, it is often necessary to preload a small block at a time from the input arrays. All the threads then need to wait until everyone has finished preloading before doing the computation on the shared memory.

Synchronization is then required again after the computation to ensure all threads have finished with the data in shared memory before overwriting it in the next loop iteration.

The function to synchronized threads is:

```cuda.syncthreads()```

This function will synchronize all threads in the same thread block. This function implements the same pattern as barriers in traditional multi-threaded programming and the ```MPI.Barrier()``` function. The program will wait until all threads in the block call the function, at which point it returns control to all its callers.

## Improved matrix multiplication
The following example shows how shared memory can be used when performing matrix multiplication.

In this example, each thread block is responsible for computing a square sub-matrix of C and each thread for computing an element of the sub-matrix. The sub-matrix is equal to the product of a square sub-matrix of A (sA) and a square sub-matrix of B (sB). In order to fit into the device resources, the two input matrices are divided into as many square sub-matrices of dimension TPB as necessary, and the result computed as the sum of the products of these square sub-matrices.

Each product is performed by first loading sA and sB from global memory to shared memory, with one thread loading each element of each sub-matrix. Once sA and sB have been loaded, each thread accumulates the result into a register (tmp). Once all the products have been calculated, the results are written to the matrix C in global memory.

By blocking the computation this way, we can reduce the number of global memory accesses since A is now only read B.shape[1] / TPB times and B is read A.shape[0] / TPB times.

<img align="left"  src="https://i.ibb.co/5xZ5cys/improve-matmul.png" alt="Drawing" style="width: 400px;"/>

In [9]:
from numba import cuda, float32
import numpy
import math

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2) # 2 = 2D grid
    print('hello')
    print(x,y)
    
    tx = cuda.threadIdx.x # position of the thread in single block
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]     # ty + i * TPB is the global idx
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads() # wait to make sA, sB full before calculation

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

# The data array
A = numpy.full((TPB*2, TPB*3), 3, float) # [32 x 48] matrix containing all 3's
B = numpy.full((TPB*3, TPB*1), 4, float) # [48 x 16] matrix containing all 4's

A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((TPB*2, TPB*1)) # [32 x 16] matrix result

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
res = C_global_mem.copy_to_host()

print(res)

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[[576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576